In [1]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk

  Cloning https://github.com/mimoralea/gym-walk to c:\users\sec\appdata\local\temp\pip-install-3g85j605\gym-walk_4568854afbc5457b93d04a3aaba02c4e
  Resolved https://github.com/mimoralea/gym-walk to commit b915b94cf2ad16f8833a1ad92ea94e88159279f5
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk 'C:\Users\SEC\AppData\Local\Temp\pip-install-3g85j605\gym-walk_4568854afbc5457b93d04a3aaba02c4e'

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import gym
import numpy as np
import random

# Fix global seeds for reproducibility
random.seed(123)
np.random.seed(123)

In [2]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k: v for k, v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        # If every transition from state s is terminal, leave blank
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0:
            print("|")

In [3]:

def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), f"{np.round(v, prec)}".rjust(6), end=" ")
        if (s + 1) % n_cols == 0:
            print("|")


In [4]:
def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    successes = 0
    # Seed action space once (optional, for reproducibility if sampling actions)
    env.action_space.seed(123)

    for episode in range(n_episodes):
        # Seed the environment at each reset for reproducible rollouts
        reset_return = env.reset(seed=123 + episode)
        if isinstance(reset_return, tuple):
            state, _ = reset_return
        else:
            state = reset_return

        done = False
        steps = 0

        while not done and steps < max_steps:
            nxt_state, _, terminated, truncated, _ = env.step(pi(state))
            done = terminated or truncated
            state = nxt_state
            steps += 1

        if state == goal_state:
            successes += 1

    return successes / n_episodes

In [5]:
def mean_return(env, pi, n_episodes=100, max_steps=200):
    returns = []
    env.action_space.seed(123)

    for episode in range(n_episodes):
        reset_return = env.reset(seed=123 + episode)
        if isinstance(reset_return, tuple):
            state, _ = reset_return
        else:
            state = reset_return

        done = False
        steps = 0
        G = 0.0

        while not done and steps < max_steps:
            nxt_state, reward, terminated, truncated, _ = env.step(pi(state))
            done = terminated or truncated
            G += reward
            state = nxt_state
            steps += 1

        returns.append(G)

    return np.mean(returns)

# FrozenLake MDP



In [6]:
env = gym.make('FrozenLake-v1')
P = env.env.P
init_state = env.reset()
goal_state = 15
LEFT, DOWN, RIGHT, UP = range(4)

In [7]:
P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [8]:
init_state

(0, {'prob': 1})

In [9]:
state, reward, terminated, truncated, info = env.step(RIGHT)
done = terminated or truncated
print(f"state:{state} - reward:{reward} - done:{done} - info:{info}")


state:1 - reward:0.0 - done:False - info:{'prob': 0.3333333333333333}


c:\Users\SEC\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [10]:
# Adversarial Policy
pi_frozenlake1 = lambda s: {
    0: RIGHT,
    1: RIGHT,
    2: RIGHT,
    3: RIGHT,
    4: RIGHT,
    5: RIGHT,
    6: RIGHT,
    7: RIGHT,
    8: RIGHT,
    9: RIGHT,
    10:RIGHT,
    11:RIGHT,
    12:RIGHT,
    13:RIGHT,
    14:RIGHT,
    15:RIGHT #Stop
}[s]
print("Name: Yuvaraj S")
print("Register Number: 212222240119")
print_policy(pi_frozenlake1, P, action_symbols=('<', 'v', '>', '^'), n_cols=4)

Name: Yuvaraj S
Register Number: 212222240119
Policy:
| 00      > | 01      > | 02      > | 03      > |
| 04      > |           | 06      > |           |
| 08      > | 09      > | 10      > |           |
|           | 13      > | 14      > |           |


In [11]:
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_frozenlake1, goal_state=goal_state)*100,
    mean_return(env, pi_frozenlake1)))

Reaches goal 2.00%. Obtains an average undiscounted return of 0.0200.


# Policy Evaluation

In [12]:
def policy_evaluation(pi, P, gamma=1.0, theta=1e-10):
    prev_V = np.zeros(len(P), dtype=np.float64)
    while True:
        V = np.zeros(len(P), dtype=np.float64)
        for s in range(len(P)):
            for prob, next_state, reward, done in P[s][pi(s)]:
                V[s] += prob * (reward + gamma * prev_V[next_state] * (not done))
        if np.max(np.abs(prev_V - V)) < theta:
            break
        prev_V = V.copy()
    return V

In [13]:
# Code to evaluate the adversarial policy
V1 = policy_evaluation(pi_frozenlake1, P)
print("Name: Yuvaraj S")
print("Register Number: 212222240119")
print_state_value_function(V1, P, n_cols=4, prec=5)

Name: Yuvaraj S
Register Number: 212222240119
State-value function:
| 00 0.0315 | 01 0.02381 | 02 0.04762 | 03    0.0 |
| 04 0.03919 |           | 06 0.09524 |           |
| 08 0.08608 | 09 0.21905 | 10 0.2381 |           |
|           | 13 0.41905 | 14 0.61905 |           |


# Policy Improvement

In [14]:
def policy_improvement(V, P, gamma=1.0):
  Q=np.zeros((len(P),len(P[0])))
  for s in range(len(P)):
      for a in range(len(P[s])):
          for prob, next_state, reward, done in P[s][a]:
              Q[s][a] += prob * (reward + gamma * V[next_state] * (not done))
  new_pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]
  return new_pi

In [15]:
pi_2 = policy_improvement(V1, P)
print("Name: Yuvaraj S")
print("Register Number: 212222240119")
print_policy(pi_2, P, action_symbols=('<', 'v', '>', '^'), n_cols=4)


Name: Yuvaraj S
Register Number: 212222240119
Policy:
| 00      < | 01      ^ | 02      < | 03      < |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |


In [16]:
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_2, goal_state=goal_state)*100,
    mean_return(env, pi_2)))

Reaches goal 77.00%. Obtains an average undiscounted return of 0.7700.


In [17]:
V2 = policy_evaluation(pi_2, P)
print("Name: Yuvaraj S")
print("Register Number: 212222240119")
print_state_value_function(V2, P, n_cols=4, prec=5)

Name: Yuvaraj S
Register Number: 212222240119
State-value function:
| 00 0.78049 | 01 0.65854 | 02 0.53659 | 03 0.26829 |
| 04 0.78049 |           | 06 0.41463 |           |
| 08 0.78049 | 09 0.78049 | 10 0.70732 |           |
|           | 13 0.85366 | 14 0.92683 |           |


In [18]:
# comparing the initial and the improved policy
if(np.sum(V1>=V2)==16):
  print("The Adversarial policy is the better policy")
elif(np.sum(V2>=V1)==16):
  print("The Improved policy is the better policy")
else:
  print("Both policies have their merits.")

The Improved policy is the better policy


# Policy Iteration

In [19]:
def policy_iteration(P, gamma=1.0, theta=1e-10):
    pi = lambda s: np.random.choice(list(P[s].keys()))  # Initialize with a random policy
    while True:
        V = policy_evaluation(pi, P, gamma, theta)  # Evaluate policy
        new_pi = policy_improvement(V, P, gamma)   # Improve policy
        if all(pi(s) == new_pi(s) for s in range(len(P))):  # Check convergence
            break
        pi = new_pi  # Update policy
    return V, pi


In [20]:
optimal_V, optimal_pi = policy_iteration(P)

In [21]:
print("Name: Yuvaraj S")
print("Register Number: 212222240119")
print('Optimal policy and state-value function (PI):')
print_policy(optimal_pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=7)


Name: Yuvaraj S
Register Number: 212222240119
Optimal policy and state-value function (PI):
Policy:
| 00      < | 01      ^ | 02      ^ | 03      ^ | 04      < |           | 06      < |
|           | 08      ^ | 09      v | 10      < |           |           | 13      > |
| 14      v |           

In [22]:
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, optimal_pi, goal_state=goal_state)*100,
    mean_return(env, optimal_pi)))

Reaches goal 81.00%. Obtains an average undiscounted return of 0.8100.


In [23]:
print("Name: Yuvaraj S")
print("Register Number: 212222240119")
print_state_value_function(optimal_V, P, n_cols=7, prec=5)

Name: Yuvaraj S
Register Number: 212222240119
State-value function:
| 00 0.82353 | 01 0.82353 | 02 0.82353 | 03 0.82353 | 04 0.82353 |           | 06 0.52941 |
|           | 08 0.82353 | 09 0.82353 | 10 0.76471 |           |           | 13 0.88235 |
| 14 0.94118 |           